In [ ]:
# Image Classifier Code

import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt
import cv2
import seaborn as sns

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [ ]:

def load_dataset(base_dir,labels):    
    "This function loads and preprocesses images."
    print('Loading...')
    
    #directories for train and test images 
    test_dir = os.path.join(base_dir,'Test')
    train_dir = os.path.join(base_dir,'Training')
    
    x_train = []
    y_train = []
    x_valid = []
    y_valid = []
    x_test = []
    y_test = []
    
    no_of_classes = len(labels)
    
    for category in labels: 
        
        #create paths for categories in labels 
        train_path = os.path.join(train_dir,category)  
        test_path = os.path.join(test_dir,category)

        #training images 
        for img in os.listdir(train_path):            
            #cv2 images are BGR images, convert them to RGB images to be compatible with matplotlib.pyplot functions 
            img_array = cv2.cvtColor(cv2.imread(os.path.join(train_path,img)),cv2.COLOR_BGR2RGB)  # convert to array
            
            #fruits-360 images are 100x100 by default, so there is no need to change their sizes 
            #img_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE)) 
            new_array = np.array(img_array)
            x_train.append(new_array)
            y_train.append(labels.index(category))
        
        #split test images into validation and test images by a ratio of 0.8
        test_image_no = len(os.listdir(test_path))
        split_no = int(test_image_no * 0.8)
        
        #validation images 
        for img in os.listdir(test_path)[:split_no]:
            img_array = cv2.cvtColor(cv2.imread(os.path.join(test_path,img)),cv2.COLOR_BGR2RGB)  # convert to array
            #img_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE)) 
            new_array = np.array(img_array)
            x_valid.append(new_array)
            y_valid.append(labels.index(category))

        #test images 
        for img in os.listdir(test_path)[split_no:]:
            img_array = cv2.cvtColor(cv2.imread(os.path.join(test_path,img)),cv2.COLOR_BGR2RGB)  # convert to array
            #img_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE)) 
            new_array = np.array(img_array)
            x_test.append(new_array)
            y_test.append(labels.index(category))
    
    #prepare label vectors/target vectors
    y_train = np_utils.to_categorical(y_train,no_of_classes)
    y_test = np_utils.to_categorical(y_test,no_of_classes)
    y_valid = np_utils.to_categorical(y_valid,no_of_classes)
    
    #convert list objects to numpy arrays 
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    
    x_valid = np.array(x_valid)    
    y_valid = np.array(y_valid)
    
    #re-scale so that all the pixel values lie within 0 to 1
    x_train = x_train.astype('float32')/255
    x_valid = x_valid.astype('float32')/255
    x_test = x_test.astype('float32')/255

    print('Done!')
    return x_train, y_train, x_valid, y_valid, x_test, y_test 

In [ ]:

base_dir = '../input/fruits/fruits-360'
labels = ['Apple Granny Smith', 'Apple Red 1', 'Avocado', 'Banana Lady Finger', 'Cocos', 'Corn', 'Eggplant', 'Mandarine', 'Pepper Green', 'Strawberry']

In [ ]:

#fruits-360 images are 100x100
IMG_SHAPE = 100

NUM_CATEGORIES = len(labels)
print('Fruit categories:', labels)
print('Fruit category no:', NUM_CATEGORIES)
          
x_train, y_train, x_valid, y_valid, x_test, y_test  = load_dataset(base_dir,labels)    
print('Train image no:', x_train.shape[0])
print('Validation image no:', x_valid.shape[0])
print('Test image no:', x_test.shape[0])

In [ ]:

#let us see one of y_train's elements 
#only one element has value 1(corresponding to its label) and others are 0.
print('y_train[0]:',y_train[0])

In [ ]:

#display sample images 
plt.imshow(x_train[0])
plt.title('Train image')
plt.savefig('sample_train_image.png')
plt.show()
index = np.argmax(y_train[0])
print('Category:', labels[index])

plt.imshow(x_valid[0])
plt.title('Validation image')
plt.savefig('sample_validation_image.png')
plt.show()
index = np.argmax(y_valid[0])
print('Category:', labels[index])

plt.imshow(x_test[0])
plt.title('Test image')
plt.savefig('sample_test_image.png')
plt.show()
index = np.argmax(y_test[0])
print('Category:', labels[index])

In [ ]:
#Build model 1
model1 = Sequential()

model1.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE, 3)))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Flatten())
model1.add(Dropout(0.2))
model1.add(Dense(512, activation='relu'))

model1.add(Dropout(0.2))
model1.add(Dense(NUM_CATEGORIES, activation='softmax'))

model1.summary()

In [ ]:

#Build model 2
model2 = Sequential()

model2.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE, 3)))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Flatten())
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))

model2.add(Dropout(0.2))
model2.add(Dense(NUM_CATEGORIES, activation='softmax'))

model2.summary()

In [ ]:

model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#Data augmentation on training set
datagen = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True,
                             rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True)

In [ ]:

epochs1 = 10
epochs2 = 5
batch_size = 32 

# fits the model on batches with real-time data augmentation
history1 = model1.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs1, shuffle = True, 
                    validation_data=(x_valid, y_valid))
                    
history2 = model2.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs2, shuffle = True, 
                    validation_data=(x_valid, y_valid))

In [ ]:

# evaluate and print test accuracy
score1 = model1.evaluate(x_test, y_test, verbose=0)
score2 = model2.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy of model #1:', score1[1])
print('\n', 'Test accuracy of model #2:', score2[1])


In [ ]:

def plot_model_performance(history,epochs,model_name):
    
    "This functions plots training and validation graphs" 
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(1,epochs + 1)
    
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.savefig(model_name + "_performance.png")
    plt.show()

plot_model_performance(history1,epochs1,"model_v1")
plot_model_performance(history2,epochs2,"model_v2")

In [ ]:

#confusion matrix for test set
y_pred2=model1.predict(x_test)
y_pred_classes2=np.argmax(y_pred2,axis=1)
y_true2=np.argmax(y_test,axis=1)

#compute confusion matrix
conf_mat2=tf.math.confusion_matrix(y_true2,y_pred_classes2)

#plot the confusion matrix
f,ax=plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat2,annot=True,fmt=".0f")
ax.set_xticklabels(labels, rotation = 'vertical')
ax.set_yticklabels(labels, rotation = 'horizontal')
plt.savefig('model_v1_confusion_matrix.png')
plt.show()


In [ ]:
#confusion matrix for test set
y_pred2=model2.predict(x_test)
y_pred_classes2=np.argmax(y_pred2,axis=1)
y_true2=np.argmax(y_test,axis=1)

#compute confusion matrix
conf_mat2=tf.math.confusion_matrix(y_true2,y_pred_classes2)

#plot the confusion matrix
f,ax=plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat2,annot=True,fmt=".0f")
ax.set_xticklabels(labels, rotation = 'vertical')
ax.set_yticklabels(labels, rotation = 'horizontal')
plt.savefig('model_v2_confusion_matrix.png')
plt.show()

In [ ]:
def plot_test_images(predictions, x_test, y_test, labels):
        
    # plot a random sample of test images, their predicted labels, and ground truth
    fig = plt.figure(figsize=(16, 9))
    
    for i, index in enumerate(np.random.choice(x_test.shape[0], size=16, replace=False)):
        
        ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
        ax.imshow(np.squeeze(x_test[index]))
        pred_index = np.argmax(predictions[index])
        true_index = np.argmax(y_test[index])
        if pred_index == true_index:
            color = 'green'
        else:
            color = 'red'
        plt.xlabel("{} {:2.0f}%".format(labels[pred_index],100*np.max(predictions[index]),labels[true_index]),color=color)
   
    plt.savefig("test_images.png")
    plt.show()


predictions = model2.predict(x_test)
plot_test_images(predictions,x_test,y_test,labels)


In [ ]:

#pip install h5py

from keras.models import model_from_json

# serialize model to JSON
model_json = model1.to_json()
with open("model_v1.json", "w") as json_file:
    json_file.write(model_json)
    
model_json = model2.to_json()
with open("model_v2.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model1.save_weights("model_v1.h5")
model2.save_weights("model_v2.h5")
print("Saved models to disk")